In [ ]:
# TODO:
# - use bold names for random variables in equations
# - use densities instead of probability symbol
# - write the Python API part

# AND ALSO:
# maybe remove the math from this notebook and write some standalone documentation for this part:
# - one document about the bayesian inversion part
# - one document about the approximation with staircase densities

# Example 0 - Working with a simple function

In this notebook we will give an overview of the API provided by `malcolm-appraiser` through a simple example.

In [ ]:
from typing import Final
import malcolm_appraiser as ma

## Forward model

We are given a forward model in the form of the following function

$$f(x,y,z) := xyz + 2yz - 3xz + 2$$

In [ ]:
def f(x: float, y: float, z: float) -> float:
    """f is a simple mathematical function we wish to optimize"""
    return x*y*z + 2*y*z - 3*x*z + 2

print(f"f(0, 0, 0) = {f(0, 0, 0)}")
print(f"f(3, 0, 3) = {f(3, 0, 3)}")
print(f"f(0, 3, 3) = {f(0, 3, 3)}")

## Problem's boundaries

For this example, we will assume that $x$, $y$ and $z$ live in the cube $[0,3]^3$.

In [ ]:
boundaries: Final = [[0,3]]*3

## Observations

We now assume that we make an observation and want to explain it using the model $f$.

We made some measurements and get the value $f(x,y,z)=0$.

However, those measurements aren't exact. There is some noise due to the precision of the sensors we used and even some random quantities at play. Fortunately, we know that the noise can be considered gaussian of mean $0$ and variance $4$.

Our observations sum-up the following way:

$$f(x,y,z) + \nu = 0$$
$$\nu \sim \mathcal{N}(0,\,4)$$

In [ ]:
from random import gauss

sigma = 2

def noisy_f(x: float, y: float, z: float) -> float:
    """noisy_f is f with added noise"""
    noise = gauss(0, sigma)
    return  f(x, y, z) + noise


print(f"f(0, 0, 0) ~ {noisy_f(0, 0, 0)}")
print(f"f(3, 0, 3) ~ {noisy_f(3, 0, 3)}")
print(f"f(0, 3, 3) ~ {noisy_f(0, 3, 3)}")

## Inversion problem

Knowing the observation we made above, we want to know how likely the latent variable $x$ is to be greater than two, that is:

$$\mathbb{P}(x > 2 | f(x,y,z)+\nu = 0)$$


### Bayes formula

Thanks to our asumption over the noise in the measurement above, we know that

$$\mathbb{P}(f(x,y,z)+\nu=v | (x,y,z)=(x_0,y_0,z_0)) = \exp{-\frac{(v-f(x_0,y_0,z_0))^2}{8}}$$

This is what we call the likelihood function (of $v$ knowing $x,y,z$), which we will write $l(x_0,y_0,z_y,v)$ below.

We are interestind in the reverse, that is
$\mathbb{P}((x,y,z)=(x_0,y_0,z_0) | f(x,y,z)+\nu=v)$, that we shall write $p(x_0,y_0,z_0,v)$ below (for posterior distribution).

Bayes formula gives

$$p(x_0,y_0,z_0,v) = \frac{l(x_0,y_0,z_0,v)\mathbb{P}((x,y,z)=(x_0,y_0,z_0))}{\iiint_{x,y,z} l(x,y,z,v) \,dx\,dy\,dz}$$


### Bayesian inference

It is not achievable to compute posterior values for every single $x_0,y_0,z_0$. What we do instead is computing expectations of functions of $x,y,z$.

$$\mathbb{E}(h(x,y,z)|f(x,y,z)+\nu = v)$$

For our problem above however, we are interested in a probability $\mathbb{P}(x > 2 | f(x,y,z)+\nu=0)$. To rewrite this as an expectation, we use the fact that for an event $A$,

$$\mathbb{E}(\mathbb{1}_A) = \mathbb{P}(A)$$
where $\mathbb{1}_A$ is the indicator function of $A$.

We use $h(x,y,z) := \mathbb{1}_{x > 2}$ and thus want to compute

$$\mathbb{E}(\mathbb{1}_{x>2} | f(x,y,z)+\nu=0) = \mathbb{P}(x > 2 | f(x,y,z)+\nu=0)$$


### Back to practice

For our practical case, we note that the Gibbs sampling method we use in this package is not sensitive to multiplicative constants. We only need to know that

$$p(x_0,y_0,z_0,v) \propto l(x_0,y_0,z_0,v)\mathbb{P}((x,y,z)=(x_0,y_0,z_0))$$

On top of that, we consider that we have no prior knowledge about $x,y,z$. Thus we take the uniform proability and $\mathbb{P}((x,y,z)=(x_0,y_0,z_0))$ is a constant. We are left with

$$p(x_0,y_0,z_0,v) \propto l(x_0,y_0,z_0,v)$$

We now want to compute an approximate value for

$$\mathbb{E}(\mathbb{1}_{x > 2} | f(x,y,z)+\nu=0) = \iiint_{x,y,z} \mathbb{1}_{x > 2} p(x,y,z,v) \,dx\,dy\,dz$$

We will use `malcolm-appraiser` to do that.

In [ ]:
# TODO